In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import sys

sys.path.append("../")

In [ ]:
import os

os.environ["CUDA_VISIBLE_DEVICES"] = "0"
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from transformers import T5Tokenizer

In [ ]:
# [pin]
file_path = "../data/data-hard.csv"
root_path = "../data/"


df = pd.read_csv(file_path)
df["prefix"] = "clsorg"
df = df.rename({"message": "input_text", "label": "target_text"}, axis=1)
df.sample(20)

,input_text,target_text,prefix
578,"""🇷🇺#AFKS """"Биннофарм групп"""" (входит в АФК С...",26-4,clsorg
5352,🇷🇺#YNDX Санкции вынудили отечественных разраб...,236-4,clsorg
4037,​​🟢 ИТОГИ ДНЯ. Российский рынок перешел к рост...,100-4;111-4;223-4,clsorg
1875,AFLT +10% 🙈,32-3,clsorg
776,"""🇷🇺#RASP """"Распадская"""" планирует увеличить го...",129-4,clsorg
4075,​​🟢 ТОП российских акций: обновление подборки ...,47-5;152-5,clsorg
2554,Какие акции предпочитают телеграм-каналы на эт...,4-4;100-4;142-4;219-4,clsorg
2752,Нефть пробила $80. Российская нефтянка продает...,111-4;112-4,clsorg
241,"""Москва. 3 апреля. INTERFAX.RU - Скидка россий...",112-3,clsorg
1908,Long YNDX?,236-4,clsorg


In [ ]:
# m_name = "t5-small"
m_name = "cointegrated/rut5-small"
tokenizer = T5Tokenizer.from_pretrained(m_name)

In [ ]:
BATCH_SIZE = 64
EPOCHS = 10
train_df, test_df = train_test_split(df, test_size=0.25, random_state=42)

In [ ]:
import torch
from transformers import T5ForConditionalGeneration

# m_name = "t5-small"
# trained_model = T5ForConditionalGeneration.from_pretrained(m_name, return_dict=True)
# state_dict = torch.load("./checkpoints/ner-v5.ckpt")["state_dict"]
# state_dict = {k.partition("model.")[2]: v for k, v in state_dict.items()}
# trained_model.load_state_dict(state_dict)
# trained_model.save_pretrained("./pretrained")

In [ ]:
m_name = "./pretrained-rut5"
trained_model = T5ForConditionalGeneration.from_pretrained(
    m_name, return_dict=True, torch_dtype=torch.float16
)
trained_model.cuda();

In [ ]:
import onnxruntime as rt
from optimum.onnxruntime import ORTModelForSeq2SeqLM

print(rt.get_available_providers())

ort_model = ORTModelForSeq2SeqLM.from_pretrained(
    "./pretrained",
    provider="CUDAExecutionProvider",
    export=True,
)

In [ ]:
import torch

from src.t5.utils import evaluate_metric, generate_answer_batched

In [ ]:
# [pin]
with torch.inference_mode():
    predictions = generate_answer_batched(
        trained_model=ort_model,
        tokenizer=tokenizer,
        data=test_df[:1200],
        batch_size=64,
        n_beams=1,
        max_length=386,
    )

  0%|          | 0/19 [00:00<?, ?it/s]

/home/worker/workspace/hakaton-gagarin-sentiment_interface/.conda/lib/python3.11/site-packages/transformers/generation/configuration_utils.py:453: UserWarning: `num_beams` is set to 1. However, `early_stopping` is set to `True` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `early_stopping`.
  warnings.warn(


In [ ]:
# [pin]
with torch.inference_mode(), torch.cuda.amp.autocast():
    predictions = generate_answer_batched(
        trained_model=trained_model,
        tokenizer=tokenizer,
        data=test_df[:1200],
        batch_size=64,
        num_beams=1,
        max_source_length=386,
    )

100%|██████████| 19/19 [00:14<00:00,  1.33it/s]


In [ ]:
ldf = test_df.copy()[:1200]
ldf["predictions"] = predictions
ldf[["tcomp", "tsent"]] = (
    ldf["target_text"].str.split(";", expand=True)[0].str.split("-", expand=True)
)
ldf[["pcomp", "psent"]] = (
    ldf["predictions"].str.split(";", expand=True)[0].str.split("-", expand=True)
)

In [ ]:
# [pin]

evaluate_metric(
    company_predictions=ldf["pcomp"].tolist(),
    company_labels=ldf["tcomp"].tolist(),
    sentiment_predictions=ldf["psent"].tolist(),
    sentiment_labels=ldf["tsent"].tolist(),
)

{'total': 58.241360011302056, 'f1': 0.6373272002260412, 'accuracy': 0.5275}